LIBRARY:

In [1]:
import numpy as np
import pandas as pd

FUNCTION:

In [2]:
def size_to_float(y):
    if (y != y.rstrip('M')):
        return float(y.rstrip('M'))*1024
    if(y != y.rstrip('k')):
        return float(y.rstrip('k'))    
    if(y == 'Varies with device'):
        return np.nan
    
def installs_to_float(y): 
    if(y == 'Free'):
        return np.nan
    return float(y.rstrip('+'))

def price_to_float(y):
    if(y=='Everyone'):
        return np.nan
    return float(y.lstrip('$'))

def word_in_review(translated_review):
    return len(translated_review.split())


this cell contains project tasks, from number 1 to number 5, and other fix to the dataSources in order to run next project tasks

In [3]:
#Starting from the Google Play Store dataset
gpApp = pd.read_csv("/home/man01/Desktop/FondamentiCS/googleplaystore.csv")
gpReview = pd.read_csv("/home/man01/Desktop/FondamentiCS/googleplaystore_user_reviews.csv")

# apply kylobyte function on gpstore.Size column, removing 'M' or 'k' from the string value and transforming MEGABYTE to KILOBYTE
#1 
#Also converting gpApp.Size from string to float
gpApp.Size = gpApp.Size.map(size_to_float)

#removing '+' and ',' from the string value, replace Free value with np.nan
#2
#Converting Installs value from string to float
gpApp['Installs'] = gpApp['Installs'].str.replace(",", "")
gpApp.Installs = gpApp.Installs.map(installs_to_float)

#3Transform “Varies with device” into a missing value
gpApp = gpApp.replace('Varies with device', np.nan)

#removing all character after a ' 'character
#4
#Convert Current Ver and Android Ver into a dotted number (e.g. 4.0.3 or 4.2)
#fixing 4.0.4W case
gpApp['Android Ver'] = gpApp['Android Ver'].str.split(' ').str[0]
gpApp["Android Ver"] = gpApp["Android Ver"].str.replace("W", "").str.strip()

#5
#removing duplicates
oldSize = gpApp.count()['App']
#gpApp[gpApp.duplicated()].size
gpApp = gpApp.drop_duplicates()
gpApp = gpApp.reset_index(drop=True)
newSize = gpApp.count()['App']

# For each category, compute the number of apps
# For each category, compute the average rating
gpApp.Rating[gpApp.Rating>5] =np.nan
gpApp["Category"] = gpApp["Category"].str.replace("1.9", "").str.strip()

#Create two dataframes: one for the genres and one bridging apps and genres. 
#So that, for instance, the app Pixel Draw - Number Art Coloring Book appears twice in the bridging table, 
#once for Art & Design, once for Creativity
gpApp["Genres"] = gpApp["Genres"].str.replace("February 11, 2018", "").str.strip()

#For each genre, create a new column of the original dataframe. 
#The new columns must have boolean values (True if the app has a given genre)
gpApp['Genres'] = gpApp['Genres'].str.split(";")

#For each app, compute the approximate income, obtain as a product of number of installs and price.
gpApp.Price=gpApp.Price.map(price_to_float)

#For each app, compute its longest review



/usr/lib/python3/dist-packages/ipykernel_launcher.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [4]:
#6 VV
#For each category, compute the number of apps
#7 VV
#For each category, compute the average rating
for cat in gpApp.Category.unique():
    gpCat = gpApp[gpApp.Category==cat]
    print("Category : ",cat," app n° : ", gpCat.Category.count()," Rating avg : ", gpCat.Rating.mean())


Category :  ART_AND_DESIGN  app n° :  65  Rating avg :  4.358064516129032
Category :  AUTO_AND_VEHICLES  app n° :  85  Rating avg :  4.19041095890411
Category :  BEAUTY  app n° :  53  Rating avg :  4.2785714285714285
Category :  BOOKS_AND_REFERENCE  app n° :  230  Rating avg :  4.3474576271186445
Category :  BUSINESS  app n° :  427  Rating avg :  4.102592592592592
Category :  COMICS  app n° :  60  Rating avg :  4.155172413793103
Category :  COMMUNICATION  app n° :  366  Rating avg :  4.151465798045603
Category :  DATING  app n° :  196  Rating avg :  3.9716981132075473
Category :  EDUCATION  app n° :  130  Rating avg :  4.375968992248062
Category :  ENTERTAINMENT  app n° :  111  Rating avg :  4.136036036036035
Category :  EVENTS  app n° :  64  Rating avg :  4.435555555555556
Category :  FINANCE  app n° :  360  Rating avg :  4.127444794952682
Category :  FOOD_AND_DRINK  app n° :  124  Rating avg :  4.164150943396227
Category :  HEALTH_AND_FITNESS  app n° :  306  Rating avg :  4.261450381

In [5]:
#8 VV
#Create two dataframes: one for the genres and one bridging apps and genres. 
#So that, for instance, the app Pixel Draw - Number Art Coloring Book appears twice in the bridging table, 
#once for Art & Design, once for Creativity
gen = pd.DataFrame(gpApp['Genres'].str[0].unique(), columns={'Genre'})
ag1 = pd.DataFrame({'App':gpApp.App , 
              'Genre':gpApp['Genres'].str[0]})
ag2 = pd.DataFrame({'App':gpApp.App , 
              'Genre':gpApp['Genres'].str[1]})
appGen = pd.concat([ag1,ag2[ag2.Genre.notnull()]])

print(appGen[appGen.App=='Pixel Draw - Number Art Coloring Book'])


                                     App         Genre
4  Pixel Draw - Number Art Coloring Book  Art & Design
4  Pixel Draw - Number Art Coloring Book    Creativity


In [6]:
#9 VV
#For each genre, create a new column of the original dataframe. 
#The new columns must have boolean values (True if the app has a given genre)
dGen = pd.get_dummies(gpApp['Genres'].apply(pd.Series).stack()).sum(level=0)
gpApp = pd.concat([gpApp, dGen], axis=1)
gpApp


,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,...,Simulation,Social,Sports,Strategy,Tools,Travel & Local,Trivia,Video Players & Editors,Weather,Word
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,19456.0,10000.0,Free,0.0,Everyone,[Art & Design],...,0,0,0,0,0,0,0,0,0,0
1,Coloring book moana,ART_AND_DESIGN,3.9,967,14336.0,500000.0,Free,0.0,Everyone,"[Art & Design, Pretend Play]",...,0,0,0,0,0,0,0,0,0,0
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,4.7,87510,8908.8,5000000.0,Free,0.0,Everyone,[Art & Design],...,0,0,0,0,0,0,0,0,0,0
3,Sketch - Draw & Paint,ART_AND_DESIGN,4.5,215644,25600.0,50000000.0,Free,0.0,Teen,[Art & Design],...,0,0,0,0,0,0,0,0,0,0
4,Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3,967,2867.2,100000.0,Free,0.0,Everyone,"[Art & Design, Creativity]",...,0,0,0,0,0,0,0,0,0,0
5,Paper flowers instructions,ART_AND_DESIGN,4.4,167,5734.4,50000.0,Free,0.0,Everyone,[Art & Design],...,0,0,0,0,0,0,0,0,0,0
6,Smoke Effect Photo Maker - Smoke Editor,ART_AND_DESIGN,3.8,178,19456.0,50000.0,Free,0.0,Everyone,[Art & Design],...,0,0,0,0,0,0,0,0,0,0
7,Infinite Painter,ART_AND_DESIGN,4.1,36815,29696.0,1000000.0,Free,0.0,Everyone,[Art & Design],...,0,0,0,0,0,0,0,0,0,0
8,Garden Coloring Book,ART_AND_DESIGN,4.4,13791,33792.0,1000000.0,Free,0.0,Everyone,[Art & Design],...,0,0,0,0,0,0,0,0,0,0
9,Kids Paint Free - Drawing Fun,ART_AND_DESIGN,4.7,121,3174.4,10000.0,Free,0.0,Everyone,"[Art & Design, Creativity]",...,0,0,0,0,0,0,0,0,0,0


In [7]:
#10 VV
#For each genre, compute the average rating. What is the genre with highest average?
#maxAverage = maxR
for g in gen.Genre.unique():
    gpGen = gpApp[gpApp.Genres.apply(lambda x: g in x)]
    print("Genre Rating : ", g,round(gpGen.Rating.mean(),2))
    if g == gen.Genre.unique()[0]:
        maxR = [g,gpGen.Rating.mean()]
    if gpGen.Rating.mean()>maxR[1]:
        maxR = (g,gpGen.Rating.mean())

print("highest avg Genre : ", maxR)

Genre Rating :  Art & Design 4.35
Genre Rating :  Auto & Vehicles 4.19
Genre Rating :  Beauty 4.28
Genre Rating :  Books & Reference 4.35
Genre Rating :  Business 4.1
Genre Rating :  Comics 4.16
Genre Rating :  Communication 4.15
Genre Rating :  Dating 3.97
Genre Rating :  Education 4.3
Genre Rating :  Entertainment 4.1
Genre Rating :  Events 4.44
Genre Rating :  Finance 4.13
Genre Rating :  Food & Drink 4.16
Genre Rating :  Health & Fitness 4.26
Genre Rating :  House & Home 4.16
Genre Rating :  Libraries & Demo 4.18
Genre Rating :  Lifestyle 4.1
Genre Rating :  Adventure 4.22
Genre Rating :  Arcade 4.31
Genre Rating :  Casual 4.18
Genre Rating :  Card 4.1
Genre Rating :  Action 4.28
Genre Rating :  Strategy 4.25
Genre Rating :  Puzzle 4.38
Genre Rating :  Sports 4.24
Genre Rating :  Music 4.24
Genre Rating :  Word 4.4
Genre Rating :  Racing 4.2
Genre Rating :  Simulation 4.17
Genre Rating :  Board 4.3
Genre Rating :  Trivia 4.04
Genre Rating :  Role Playing 4.28
Genre Rating :  Educat

In [8]:
#11 VV
#For each app, compute the approximate income, obtain as a product of number of installs and price.
appIncome=round(gpApp.Price*gpApp.Installs,3)
for inc in range(appIncome.count()):
    if(appIncome[inc]>0):
        print(inc, gpApp['App'][inc], appIncome[inc])
    if(inc>1000):
        break

233 TurboScan: scan documents and receipts in PDF 499000.0
234 Tiny Scanner Pro: PDF Doc Scan 499000.0
390 Puffin Browser Pro 399000.0
429 Moco+ - Chat, Meet People 39900.0
430 Calculator 6990.0
431 Truth or Dare Pro 74.5
432 Private Dating, Hide App- Blue for PrivacyHider 299.0
433 Ad Blocker for SayHi 399.0
434 AMBW Dating App: Asian Men Black Women Interracial 799.0
507 Moco+ - Chat, Meet People 39900.0
741 Sago Mini Hat Maker 3990.0
742 Fuzzy Numbers: Pre-K Number Foundation 5990.0
743 Toca Life: City 1995000.0
744 Toca Life: Hospital 399000.0
847 My Talking Pet 499000.0
853 Meme Generator 299000.0


In [9]:
#12 VV
#For each app, compute its minimum and maximum Sentiment_polarity
revii = gpReview.App[1]
count=0
for rap in gpReview.App.unique():
    if(revii!=rap):
        count=count+1
        revii=rap   
    gpRap = gpReview[gpReview.App==rap]
    print("App : ", rap," Min SentimentP : ", round(gpRap.Sentiment_Polarity.min(),2)," Max SentimentP : ", round(gpRap.Sentiment_Polarity.max(),2))
    if (count>=10):
        break

App :  10 Best Foods for You  Min SentimentP :  -0.8  Max SentimentP :  1.0
App :  104 找工作 - 找工作 找打工 找兼職 履歷健檢 履歷診療室  Min SentimentP :  -0.11  Max SentimentP :  0.91
App :  11st  Min SentimentP :  -1.0  Max SentimentP :  1.0
App :  1800 Contacts - Lens Store  Min SentimentP :  -0.3  Max SentimentP :  0.84
App :  1LINE – One Line with One Touch  Min SentimentP :  -0.82  Max SentimentP :  1.0
App :  2018Emoji Keyboard 😂 Emoticons Lite -sticker&gif  Min SentimentP :  -0.8  Max SentimentP :  1.0
App :  21-Day Meditation Experience  Min SentimentP :  -0.27  Max SentimentP :  0.59
App :  2Date Dating App, Love and matching  Min SentimentP :  -0.65  Max SentimentP :  1.0
App :  2GIS: directory & navigator  Min SentimentP :  -0.38  Max SentimentP :  1.0
App :  2RedBeans  Min SentimentP :  -0.8  Max SentimentP :  1.0
App :  2ndLine - Second Phone Number  Min SentimentP :  -0.78  Max SentimentP :  1.0
